In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ln -s /content/drive/My\ Drive /mydrive
!nvidia-smi

Mounted at /content/drive
Tue Nov  3 10:05:28 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                              

In [2]:
%%time
%%bash
# INSTALL MINICONDA
MINICONDA_INSTALLER_SCRIPT=Miniconda3-latest-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget -q https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX
which conda

# INSTALL system packages
apt-get update && apt-get install -qq bc tree sox ffmpeg libsndfile1 libsox-fmt-mp3


# PIP INSTALL NeMo
conda create -n nemo python=3.8 -y
conda init bash
source activate nemo
pip -q install torch==1.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
cd /mydrive/NeMo/ && pip -q install -r requirements/requirements.txt
cd /mydrive/NeMo/ && pip -q install -e .[asr]
pip -q install wandb torchaudio==0.6.0
pip -q install "util@git+https://git@github.com/dertilo/util.git#egg=util"

source activate nemo
python -c "import torch; print(torch.cuda.is_available())"

# INSTALL ctc-decoder
source activate nemo
apt-get install -qq swig
cd /mydrive/NeMo
git clone https://github.com/NVIDIA/OpenSeq2Seq -b ctc-decoders
mv OpenSeq2Seq/decoders .
rm -rf OpenSeq2Seq
cd decoders
sh ./setup.sh > decoder_setup.log

# EXTRACT DATA
rm -rf /content/corpora
source activate nemo
python
import tarfile
from pathlib import Path
import os

corpora_dir = "/mydrive/data/asr_data/ENGLISH"
zips = list(Path(corpora_dir).rglob("*processed_mp3.tar.gz"))
dest_dir = "/content/corpora/"
os.makedirs(dest_dir,exist_ok=True)
for zipfile in zips:
    with tarfile.open(zipfile, mode="r:gz") as tar:
        tar.extractall(dest_dir)


PREFIX=/usr/local
Unpacking payload ...
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - ca-certificates==2020.1.1=0
    - certifi==2020.4.5.1=py38_0
    - cffi==1.14.0=py38he30daa8_1
    - chardet==3.0.4=py38_1003
    - conda-package-handling==1.6.1=py38h7b6447c_0
    - conda==4.8.3=py38_0
    - cryptography==2.9.2=py38h1ba5d50_0
    - idna==2.9=py_1
    - ld_impl_linux-64==2.33.1=h53a641e_7
    - libedit==3.1.20181209=hc058e9b_0
    - libffi==3.3=he6710b0_1
    - libgcc-ng==9.1.0=hdf63c60_0
    - libstdcxx-ng==9.1.0=hdf63c60_0
    - ncurses==6.2=he6710b0_1
    - openssl==1.1.1g=h7b6447c_0
    - pip==20.0.2=py38_3
    - pycosat==0.6.3=py38h7b6447c_1
    - pycparser==2.20=py_0
    - pyopenssl==19.1.0=py38_0
    - pysocks==1.7.1=py38_0
    - python==3.8.3=hcff3b4d_0
    - readline==8.0=h7b6447c_0
    - requests==2.23.0=py38_0
    - ruamel_yaml==0.15.87=py38h7b6447c_0
    - setupt

                                                                                        

==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.9.1

Please update conda by running

    $ conda update -n base -c defaults conda


Cloning into 'OpenSeq2Seq'...
Checking out files: 100% (519/519), done.
Cloning into 'kenlm'...
Checking out files: 100% (299/299), done.
--2020-11-03 10:09:10--  http://www.openfst.org/twiki/pub/FST/FstDownload/openfst-1.6.3.tar.gz
Resolving www.openfst.org (www.openfst.org)... 206.196.111.47
Connecting to www.openfst.org (www.openfst.org)|206.196.111.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1213465 (1.2M) [application/x-gzip]
Saving to: ‘openfst-1.6.3.tar.gz’

     0K .......... .......... .......... .......... ..........  4% 1.08M 1s
    50K .......... .......... .......... .......... ..........  8% 1.04M 1s
   100K .......... .......... .......... .......... .......... 12% 2.04M 1s
 

CPU times: user 84.1 ms, sys: 28.6 ms, total: 113 ms
Wall time: 10min 20s


In [8]:
%%bash
#du -sh /content/corpora

zcat /mydrive/data/asr_data/ENGLISH/kenlm_3_089.arpa.gz > /content/kenlm.arpa
head -n 10 /content/kenlm.arpa


\data\
ngram 1=200003
ngram 2=3863484
ngram 3=7323847

\1-grams:
-7.625509	<unk>	0
0	<s>	-1.9915925
-2.097004	</s>	0


In [9]:
%%time
%%bash
# INFERENCE on MANIFEST
source activate nemo
python -c "import torch; print(torch.cuda.is_available())"

python /mydrive/lfc/run_batch_inference.py --corpora_dir /content/corpora --arpa "/mydrive/data/asr_data/ENGLISH/kenlm_3_089.arpa.gz" --results_dir "/mydrive/data/asr_data/results/kenlm_3_089_mp3"


True
Using NGC cloud ASR model QuartzNet5x5LS-En
[NeMo I 2020-11-03 10:36:35 cloud:55] Found existing object /root/.cache/torch/NeMo/NeMo_1.0.0b2/QuartzNet5x5LS-En/cf17856f9aa9ba75ddf7881277884caa/QuartzNet5x5LS-En.nemo.
[NeMo I 2020-11-03 10:36:35 cloud:61] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.0.0b2/QuartzNet5x5LS-En/cf17856f9aa9ba75ddf7881277884caa/QuartzNet5x5LS-En.nemo
[NeMo I 2020-11-03 10:36:35 common:399] Instantiating model from pre-trained checkpoint
[NeMo I 2020-11-03 10:36:36 features:241] PADDING: 16
[NeMo I 2020-11-03 10:36:36 features:258] STFT using torch
[NeMo I 2020-11-03 10:36:40 modelPT:251] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0b2/QuartzNet5x5LS-En/cf17856f9aa9ba75ddf7881277884caa/QuartzNet5x5LS-En.nemo.
[NeMo I 2020-11-03 10:37:23 collections:173] Dataset loaded with 132553 files totalling 467.49 hours
[NeMo I 2020-11-03 10:37:23 collections:174] 0 files were filtered totalling 0.00 hours
USING GPU!
Got 

[NeMo W 2020-11-03 10:36:35 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToCharDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-11-03 10:36:35 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToBPEDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-11-03 10:36:35 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_text.AudioLabelDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-11-03 10:36:35 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_text._TarredAudioToTextDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-11-03 10:36:35 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_text.TarredAudioToCharDataset'> is experimental,

CPU times: user 482 ms, sys: 81.4 ms, total: 563 ms
Wall time: 1h 2min 37s


In [9]:
%%time
%%bash
# GET KENLM TRAIN DATA
wget -q http://www.openslr.org/resources/11/librispeech-lm-norm.txt.gz
cp librispeech-lm-norm.txt.gz /mydrive/data/asr_data/librispeech-lm-norm.txt.gz

zcat /mydrive/data/asr_data/librispeech-lm-norm.txt.gz > /content/lm_corpus.txt

wc -l lm_corpus.txt

40418261 lm_corpus.txt
CPU times: user 5.38 ms, sys: 7.35 ms, total: 12.7 ms
Wall time: 1min 18s


In [11]:
%%time
%%bash
# BUILD KENLM
git clone https://github.com/kpu/kenlm.git
cd kenlm
mkdir -p build
cd build
cmake ..
make -j 4

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Boost version: 

Cloning into 'kenlm'...


CPU times: user 23.2 ms, sys: 8.18 ms, total: 31.4 ms
Wall time: 1min 25s


In [15]:
%%time
%%bash
# TRAIN KENLM
cd kenlm/build
bin/lmplz -o 3 -S 80% -T --prune 0 3 /tmp </content/lm_corpus.txt >librispeech.arpa
cat librispeech.arpa | gzip > /mydrive/data/asr_data/ENGLISH/librispeech.arpa.gz

Process is terminated.
CPU times: user 62.5 ms, sys: 29.3 ms, total: 91.8 ms
Wall time: 16min 14s


In [ ]:
%%time
%%bash
# INFERENCE single FILE
source activate nemo
python

import nemo.collections.asr as nemo_asr
import numpy as np

asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet5x5LS-En")

# softmax implementation in NumPy
def softmax(logits):
    e = np.exp(logits - np.max(logits))
    return e / e.sum(axis=-1).reshape([logits.shape[0], 1])


# let's do inference once again but without decoder
data_path = "/content/dev-other_processed_wav"
files = [f"{data_path}/content_SPANISH_raw_dev-other_LibriSpeech_dev-other_116_288045_116-288045-0000.wav"]

transcript = asr_model.transcribe(paths2audio_files=files)[0]
print(f'Transcript: "{transcript}"')

logits = asr_model.transcribe(files, logprobs=True)[0].cpu().numpy()
probs = softmax(logits)
print(probs)